In [16]:
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 4.0.5"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
library(keras)




In [3]:
use_condaenv("keras-tf", required = T)

In [8]:
df <- read.csv('d:\\alldata\\kyphosis.csv')

In [10]:
head(df)

,Kyphosis,Age,Number,Start
,<dbl>,<int>,<int>,<int>
1,0,71,3,5
2,0,158,3,14
3,1,128,4,5
4,0,2,5,1
5,0,1,4,15
6,0,1,2,16


In [9]:
df$Kyphosis <- ifelse(df$Kyphosis=="absent",0,1)

In [11]:
library(caTools)

Warning message:
"package 'caTools' was built under R version 4.0.5"


In [12]:
splits <- sample.split(df$Kyphosis,SplitRatio=0.7)
train <- subset(df,splits==TRUE)
test <- subset(df,splits==FALSE)

In [13]:
dim(train)

[1] 57  4

In [14]:
dim(test)

[1] 24  4

In [17]:
x_train <- train %>% 
  select(-Kyphosis) %>% 
  scale()
  
y_train <- to_categorical(train$Kyphosis)

In [21]:
head(y_train)

0,1
1,0
1,0
1,0
1,0
1,0


In [22]:
x_test <- test %>% 
  select(-Kyphosis) %>% 
  scale()

y_test <- to_categorical(test$Kyphosis)

In [23]:
head(x_test)

,Age,Number,Start
1,-0.1011799,-0.6308185,-1.41904824
2,1.4297163,-0.6308185,0.44994213
8,-0.6994612,-0.6308185,0.86527332
10,-0.3123380,1.1160636,0.03461093
14,-1.3329355,-0.0485245,0.03461093
16,-1.3329355,-0.6308185,0.86527332


In [24]:
model <- keras_model_sequential() 

In [25]:
model %>% 
  layer_dense(units = 256, activation = 'relu', input_shape = ncol(x_train)) %>% 
  layer_dropout(rate = 0.4) %>% 
  layer_dense(units = 128, activation = 'relu') %>%
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 2, activation = 'sigmoid')

In [26]:
history <- model %>% compile(
  loss = 'binary_crossentropy',
  optimizer = 'adam',
  metrics = c('accuracy')
)

In [27]:
model %>% fit(
  x_train, y_train, 
  epochs = 100, 
  batch_size = 5,
  validation_split = 0.3
)

In [28]:
summary(model)

Model: "sequential"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
dense_2 (Dense)                     (None, 256)                     1024        
________________________________________________________________________________
dropout_1 (Dropout)                 (None, 256)                     0           
________________________________________________________________________________
dense_1 (Dense)                     (None, 128)                     32896       
________________________________________________________________________________
dropout (Dropout)                   (None, 128)                     0           
________________________________________________________________________________
dense (Dense)                       (None, 2)                       258         
Total params: 34,178
Trainable params: 34,178
Non-trainable params: 0
___________________

In [29]:
model %>% evaluate(x_test, y_test)

loss  accuracy 
0.6412406 0.8333333

In [34]:
predictions <- model %>% predict_classes(x_test)

Warning message in predict_classes(., x_test):
"`predict_classes()` is deprecated and and was removed from tensorflow in version 2.6.
Please update your code:
  * If your model does multi-class classification:
    (e.g. if it uses a `softmax` last-layer activation).

      model %>% predict(x) %>% k_argmax()

  * if your model does binary classification
    (e.g. if it uses a `sigmoid` last-layer activation).

      model %>% predict(x) %>% `>`(0.5) %>% k_cast("int32")
"


In [35]:
head(predictions)

[1] 0 0 0 0 0 0

In [37]:
table(factor(predictions, levels=min(test$Kyphosis):max(test$Kyphosis)),factor(test$Kyphosis, levels=min(test$Kyphosis):max(test$Kyphosis)))


   
     0  1
  0 18  3
  1  1  2